# Embeddings con LSTM



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
import random
from itertools import chain
from pprint import pprint
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [2]:
cd ..

C:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\ProyectoFinal


In [3]:
from helpers.boletines import get_sentences

In [4]:
cd Embeddings/

C:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\ProyectoFinal\Embeddings


## Definición de constantes

In [5]:
# Número de GPUs disponibles. Usar 0 para modo CPU.
ngpu = 1

# Semilla a usar en los generadores de números aleatorios
SEED = 42069
# SEED = random.randint(1, 10000) # En caso de requerir más resultados
random.seed(SEED)
torch.manual_seed(SEED)

print("Random Seed: ", SEED)

Random Seed:  42069


Selecciono el tipo de dispositivo a utilizar (gpu o cpu)

In [6]:
# Decide si queremos correr en gpu o cpu
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
device

device(type='cuda', index=0)

## Obtener corpus

In [7]:
uri = open('./../mongo_uri.txt', 'r', encoding='utf-8').read()

In [8]:
sentences = get_sentences(uri)

JAMK
1. José_Antonio_Meade fue recibido en Culiacán por cientos de simpatizantes sinaloenses .
377. El aspirante presidencial se comprometió a mejorar la infraestructura carretera e hidráulica en la entidad .
753. “ Si tuviera que resumir en una idea el país por el que apuesto para los próximos seis años , lo resumiría en tres palabras : un México líder ” , indicó .
1129. De_acuerdo_con diversas estimaciones , las armas en poder de la delincuencia organizada en México podrían alcanzar el número de 1.5 millones de piezas , esto es 3 veces el arsenal con el que cuenta , por_ejemplo , el ejército de Guatemala .
1505. Expresó que , sin_ninguna_duda y con toda certeza , logrará la victoria el 1_de_julio .
1881. Dijo que con Mikel_Arriola va por una Ciudad_de_México donde abramos la llave y salga agua , que no sea extorsionada por delegados que condicionan el agua a la militancia política , una ciudad que no tandee el agua , donde el derecho a el agua sea vigente , se eviten las fugas y se e

Desenvuelvo los grupos de enunciados en un único arreglo con todos los enunciados de todos los candidatos.

In [9]:
corpus = list(chain(*sentences.values()))

Paso los tokens a minúsculas para reducir el tamaño del vocabulario

In [10]:
sentences = [[w.lower() for w in sent] for sent in corpus]
num_sents = len(sentences)
print(f'El corpus consta de {num_sents} enunciados.')

El corpus consta de 6889 enunciados.


Selecciono k oraciones de prueba para el modelo

In [11]:
# k = 6889
# sentences = random.sample(corpus, k=k)
print('\n'.join([' '.join(sent) for sent in sentences[::num_sents//10]]))

josé_antonio_meade fue recibido en culiacán por cientos de simpatizantes sinaloenses .
josé_antonio_meade hizo notar que “ 7_de_cada_10 homicidios que se incrementaron en los últimos dos años , se dieron en estados gobernados por el frente ” .
de_cara_a las últimas cinco semanas de contienda , josé_antonio_meade pidió el apoyo de los campechanos para salir a las calles a dar la batalla por méxico y convencer a el electorado de que su proyecto de gobierno es la mejor opción .
“ no tengo ninguna duda , la próxima jefa_de_gobierno_de_la_ciudad_de_méxico será alejandra_barrales ” , afirmó el candidato a la presidencia_de_la_república , ricardo_anaya , a el respaldar el programa , proyecto y candidatura de quien el día de hoy se presentó ante miles de mujeres con propuestas , para pedir les su apoyo de_cara_a la elección de el próximo_1_de_julio .
por lo anterior , reiteró su rechazo a la amnistía y perdón a criminales : “ sí a la verdad , para que haya justicia , y sí a la justicia para al

## Indexar Corpus

Cuento los tokens en todas las oraciones e imprimo los 40 más frecuentes.

In [12]:
%%time
freq = sum([Counter(sent) for sent in sentences], Counter())

Wall time: 26 s


In [13]:
n_tipos = len(freq.keys())
print(f'Número de tipos: {n_tipos}')
print(freq.most_common()[::max(n_tipos//25, 1)])

Número de tipos: 13849
[(',', 19701), ('guerrero', 46), ('colima', 21), ('competitividad', 13), ('apoye', 9), ('narcotráfico', 7), ('500', 5), ('tomado', 4), ('directivos', 3), ('jefa_de_gobierno', 3), ('dicte', 3), ('aceptado', 2), ('antiguas', 2), ('remito', 2), ('perjudica', 2), ('encima', 1), ('tendrá_lugar', 1), ('albergan', 1), ('brindando', 1), ('incurriera', 1), ('instituto_politécnico', 1), ('encuestadora', 1), ('honradamente', 1), ('declaren', 1), ('juan_carlos_plaza', 1), ('capal', 1)]


Agrego un diccionario para pasar de palabra a índice numérico

In [14]:
w_to_index = {
    w : ix
    for ix, (w, freq) in enumerate(freq.most_common())
    if freq > 1 # No toma en cuenta los hapax
}

In [15]:
vocab_size = len(w_to_index)
vocab_size

7793

In [16]:
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<UNK>'

ixBOS = vocab_size
ixEOS = vocab_size + 1
ixUNK = vocab_size + 2

In [17]:
w_to_index[BOS] = ixBOS
w_to_index[EOS] = ixEOS
w_to_index[UNK] = ixUNK

Calculo el nuevo tamaño del vocabulario después de agregar 3 tokens

In [18]:
vocab_size = len(w_to_index)
vocab_size

7796

Creo el diccionario inverso, para convertir de índices a palabras

In [19]:
index_to_w = [ w for w, ix in w_to_index.items() ]

Indexo todo el corpus

In [20]:
def w_to_index_unk(w):
    """
    Le asigna el token UNK a palabras que no aparezcan en el corpus
    """
    try:
        return w_to_index[w] 
    except KeyError:
        return ixUNK

In [21]:
sentences_ix = [
    # Le agrego el inicio y fin de caracter a los enunciados
    [ixBOS] + [ w_to_index_unk(w) for w in sent ] + [ixEOS] 
    for sent in sentences
]

X = [ np.asarray(sent[:-1]) for sent in sentences_ix ]
Y = [ np.asarray(sent[1:]) for sent in sentences_ix ] 

print('Ejemplos X')
print(X[:3])
print('Ejemplos Y')
print(Y[:3])

Ejemplos X
[array([7793,   44,  119, 1035,    8, 1998,   19, 1117,    1,  289, 5634,
          6]), array([7793,    5,    3, 1999,    1, 2000, 3203,    0,   75, 1036,    7,
       1732,    5,   21, 2844,    0,    3,   25,    1,    4,   41,   91,
          0,   44,    0, 2567,   65,  892,    5,    3, 7795,    0,  861,
          0, 4420,    1,  628,    0,    8,    4, 5635,    1,   21,   50,
        142,    2,   20, 2138,    5, 2316,   57,    3, 1446,  178,    6]), array([7793,    3,  225,   55,  119, 1035,   19, 2568,    1,  289,    2,
         10, 1733,    8,    4, 2139, 2317,    1,    4,  430, 3203,    0,
        102, 5636, 5637, 7795,   13,   21, 7795,    6])]
Ejemplos Y
[array([  44,  119, 1035,    8, 1998,   19, 1117,    1,  289, 5634,    6,
       7794]), array([   5,    3, 1999,    1, 2000, 3203,    0,   75, 1036,    7, 1732,
          5,   21, 2844,    0,    3,   25,    1,    4,   41,   91,    0,
         44,    0, 2567,   65,  892,    5,    3, 7795,    0,  861,    0,
       4420

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30)

In [23]:
print(len(X_train))
print(len(X_test))

4822
2067


Mando los vectores de entrada y salida a tensores en gpu

In [24]:
def to_pytorch_tensor(list_of_lists):
    return [
        torch.from_numpy(l).long().to(device)
        for l in list_of_lists
    ]

In [25]:
X_train = to_pytorch_tensor(X_train)
Y_train = to_pytorch_tensor(Y_train)

X_test = to_pytorch_tensor(X_test)
Y_test = to_pytorch_tensor(Y_test)

## Modelo

### 1. Capa de embedding

### 2. Capa oculta

### 3. Capa de salida

Defino las variables para la red neuronal

In [26]:
# Dimensión de entrada (one-hot), tamaño del vocabulario
D_in = vocab_size

# Dimensión de la capa de embedding
D_emb = 32 # 32

# Dimensión de la capa lstm
D_lstm = 16 # 16

# Dimensión de la capa de salida
D_out = D_in

# Épocas de entrenamiento
num_epochs = 300

# Betas para Adam
beta1 = 0.0001
beta2 = 0.99

# Learning rate
lr =  0.1

In [27]:
class Model(nn.Module):
    def __init__(self, ngpu, D_in, D_emb, D_lstm, D_out):
        super(Model, self).__init__()
        self.ngpu = ngpu
        self.embedding = nn.Embedding(num_embeddings=D_in, embedding_dim=D_emb)#, padding_idx=0)
        self.lstm = nn.LSTM(input_size=D_emb, hidden_size=D_lstm) #, bias=True)#, batch_first=True)
        self.linear = nn.Linear(in_features=D_lstm, out_features=D_out) #, bias=True)

    def forward(self, sentence):
        T = len(sentence)

        embeddings = self.embedding(sentence).view(T, 1, -1)

        lstm_out, (ht, ct) = self.lstm(embeddings)
        lstm_out = lstm_out.view(T, -1)

        preact_out = self.linear(lstm_out).view(T, -1)

        return F.log_softmax(preact_out, dim=1)

In [28]:
model = Model(ngpu, D_in, D_emb, D_lstm, D_out).to(device)
model

Model(
  (embedding): Embedding(7796, 32)
  (lstm): LSTM(32, 16)
  (linear): Linear(in_features=16, out_features=7796, bias=True)
)

In [29]:
# Inicialización de pesos
# def weights_init(m):
#     classname = m.__class__.__name__
#     if classname.find('Embedding') != -1:
#         # Regularizo los pesos
#         n = m.num_embeddings
#         y = 1.0/np.sqrt(n)
#         m.weight.data.uniform_(-y, y)
#     elif classname.find('Linear') != -1:
#         n = m.in_features
#         y = 1.0/np.sqrt(n)
#         m.weight.data.uniform_(-y, y)
#         m.bias.data.fill_(0)
#     elif classname.find('LSTM') != -1:
#         n = m.input_size
#         y = 1.0/np.sqrt(n)
#         m.weight_ih_l0.data.uniform_(-y, y)
#         m.weight_hh_l0.data.uniform_(-y, y)
#         m.bias_ih_l0.data.fill_(0)
#         m.bias_hh_l0.data.fill_(0)

In [30]:
# model.apply(weights_init)

Entropía cruzada como optmizador y SGD como optimizador

In [31]:
criterion = nn.NLLLoss()
criterion.to(device)
# criterion = nn.NLLLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(beta1, beta2))
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

### Entrenamiento

In [32]:
%%time
for epoch in tqdm(range(num_epochs)):
# for epoch in range(10):
# for epoch in range(num_epochs):
    epoch_loss = 0
    # for x, y in zip(X_train, Y_train):
    for x, y in zip(X_train, Y_train):
        if len(x) == 0:
            tqdm.write('Sentencia vacía')
            continue
        # Limpiamos gradientes acumulados
        model.zero_grad()

        # Forward
        out = model(x)
     
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        # break
    # tqdm.write(f'Epoch: {epoch:>12} Loss: {loss}')
    # break
    if epoch%20 == 0:
        tqdm.write(f'Coste después de {epoch+1} épocas: {epoch_loss}')

Coste después de 1 épocas: 29586.33521795273
Coste después de 21 épocas: 23045.75836133957
Coste después de 41 épocas: 21993.18262529373
Coste después de 61 épocas: 21330.63555073738
Coste después de 81 épocas: 20866.377475857735
Coste después de 101 épocas: 20508.08817899227
Coste después de 121 épocas: 20226.031289815903
Coste después de 141 épocas: 19998.21378850937
Coste después de 161 épocas: 19797.864720582962
Coste después de 181 épocas: 19629.004155278206
Coste después de 201 épocas: 19483.502443432808
Coste después de 221 épocas: 19351.651683568954
Coste después de 241 épocas: 19232.752661585808
Coste después de 261 épocas: 19126.964255928993
Coste después de 281 épocas: 19034.05273604393

Wall time: 1h 54min 25s


In [33]:
# list(model.named_parameters())

In [34]:
def print_ix_sentence(sentence):
    print(' '.join(index_to_w[ix] for ix in sentence.data))

In [35]:
# See what the scores are after training
with torch.no_grad():
    for sentence, y in zip(X_test[:10], Y_test[:10]):
        print('===================================================')
        print('- Probando enunciado:')
        print_ix_sentence(sentence)
        print()

        out = model(sentence)
        prediccion = torch.argmax(out, dim=1)
        print('- Predicción:')
        print_ix_sentence(prediccion)
        print
        print('- Real:')
        print_ix_sentence(y)
        print('\n')

- Probando enunciado:
<BOS> en entrevista , el candidato por la coalición “ juntos haremos historia ” a la presidencia_de_méxico deseó que los gobernadores se <UNK> en el sentido de que se respetará la voluntad de los ciudadanos y de que se va a respetar a quien <UNK> triunfador en los procesos electorales .

- Predicción:
“ otro , andrés_manuel_lópez_obrador candidato de la coalición “ juntos haremos historia ” en el presidencia_de_la_república , que el <UNK> de <UNK> a la país de la se <UNK> a <UNK> de la mexicanos , la el se <UNK> a ganar la el <UNK> , , el últimos de , <EOS>
- Real:
en entrevista , el candidato por la coalición “ juntos haremos historia ” a la presidencia_de_méxico deseó que los gobernadores se <UNK> en el sentido de que se respetará la voluntad de los ciudadanos y de que se va a respetar a quien <UNK> triunfador en los procesos electorales . <EOS>


- Probando enunciado:
<BOS> “ vamos a ganar porque aquí está la estructura , porque hay perfil , propuestas , sentid

In [45]:
torch.save(model.state_dict(), 'model-seed42069')

In [36]:
# params = list(model.named_parameters())
# list(model.named_parameters())

In [37]:
# embs = model.embedding.weight

In [38]:
# npembs = embs.detach().to('cpu').numpy()

In [39]:
# npembs

In [40]:
# npembs[:,0]

In [41]:
# npembs[:,1]

In [42]:
# plt.scatter(npembs[:,0], npembs[:,1])
# r=0
# for label,x,y in zip(index_to_w, npembs[:,0], npembs[:,1]):
#     plt.annotate(label, xy=(x,y), xytext=(-1,1), textcoords='offset points', ha='center', va='bottom')
#     r+=1
# plt.show()

In [43]:
# index_to_w

In [44]:
# params[0][1]